In [145]:
using LinearAlgebra

### Função de Taylor

In [12]:
"""
RESUMO: Esse método recebe um ponto a e a imagem de a em uma funcao que nao sabemos calcular diretamente.
        A partir dessas informacoes e das derivadas, aproximamos o valor da imagem de x nessa funcao usando o método
        de Taylor.
ENTRADA: Float Fa (imagem da funcao em a), Vetor 'derivadas', Float a, Float x
SAÍDA: Float soma
"""
function taylor(Fa, derivadas, a, x)
    grau=length(derivadas) #grau do Taylor vai ser implícito
    soma = Fa
    for k = 1:grau
        soma += (derivadas[k] * (x - a)^k) / factorial(k)
    end
    return soma
end

taylor

#### Exemplos

In [19]:
# Exemplo 1 - log(x), a = 1
Fa = 0  # log 1
derivadas = [1, -1, 2]
a = 1.0
x = 1.5
y = taylor(Fa,derivadas,a,x)
y - log(1.5)

0.0112015585585023

In [23]:
# Exemplo 2 - sen(x), a = 0
Fa = 0  # sen(0)
derivadas = [1, 0, -1, 0, 1, 0, -1]
a = 0
x = 0.1
y = taylor(Fa,derivadas,a,x)
y - sin(0.1)

-2.7478019859472624e-15

In [25]:
# Exemplo 3 -> sqrt(x), a = 1
Fa = 1
derivadas = [0.5, -0.25, 3.0/8.0]
a = 1
x = 1.21
y = taylor(Fa,derivadas,a,x)
y - 1.1

6.631249999977662e-5

### Função Bisseção

In [6]:
"""
RESUMO: Este método pode ser usado para encontrar as raízes de uma função contínua, com base 
        no teorema do valor intermediário, tendo f(a) e f(b) com sinais opostos, tem de se ter uma raíz entre a e b
        O método consiste em dividir o intervalo no seu ponto médio c=(a+b)/2, e então verificar em qual dos dois 
        subintervalos garante-se a existência de uma raiz. Para tanto, basta verificar se  f(a)*f(c)<0. 
        Caso afirmativo, existe pelo menos uma raiz no intervalo (a,c), caso contrário garante-se a 
        existência de uma raiz no intervalo [c,b). O procedimento é, então, repetido para o subintervalo 
        correspondente à raiz até que c se aproxime a raiz com a precisão desejada.
ENTRADA: funcao, precisao, a, b
SAÍDA: raiz
"""

function bissecao(funcao, precisao, a, b ;max_tempo = 10.0, max_iter = 100)
    # Limitamos o tempo e o número de iterações, pois estamos usando while e não queremos cair em um loop infinito
    fa = f(a) # Função aplicada em a
    fb = f(b) # Função aplicada em b
    ϵ  = precisao *max(abs(fa), abs(fb)) 
    #Em caso dos valores dos pontos na função serem próximos o suficiente de 0 para não executar o código
    if abs(fa) ≤ ϵ 
        print("a é próximo de ϵ")
        return a
    elseif abs(fb)≤ ϵ 
        print("b é próximo de ϵ")
        return b
    elseif fa*fb ≥ 0
        return "ERRO: F(A) E F(B) TEM SINAIS IGUAIS"
    end
    ϵba  = precisao * abs(b-a) # Calculando o tamanho do erro que é possível, com base na precisão
    
    c = (a+b)/2 # Calculando um x que é o meio entre a e b
    fc = f(c) # Calculando a função no ponto c
    # Inicializando as variáveis de tempo e iteração
    iter =0 
    t0 = time()
    Δt = time() - t0
    
    # Será resolvido quando o módulo de b-a * precisão for menor do que b-a, ou seja a distância seja desse valor
    # Ou se o módulo de f(c) seja suficientemente perto de 0, segundo a precisão
    resolvido = (abs(fc) ≤ ϵ || abs(b-a) ≤ ϵba) 
    
    cansado = (iter ≥ max_iter || Δt ≥ max_tempo)
    # LOOP do método da bisseção
    # Vamos fazendo calculando o intervalo entre a e b até encontrar algum f(c) ≈ 0, dentro da precisão 
    # ou o intervalo entre b-a menor que a precisão
    while !(resolvido || cansado)
        if fa *fc < 0
            b = c
            fb = fc
        else
            a = c
            fa = fc
        end
        
        c = (a+b)/2
        fc = f(c)
        iter +=1
        Δt = time()-t0
        
        resolvido = (abs(fc) ≤ ϵ || abs(b-a) ≤ ϵba)
        cansado = (iter ≥ max_iter || Δt ≥ max_tempo)
    end
    if resolvido
        return c
    elseif cansado
        if iter ≥ max_iter
            return "ERRO: Chegamos ao número máximo de iterações"
        else
            return "ERRO: Chegamos ao máximo de tempo de execução"
        end
    end
end

bissecao (generic function with 1 method)

#### Exemplos

In [7]:
# Exemplo 1
f(x) = sin(x)
b = 1
a = 5
# Sabemos que sen(π) = 0
precisao = 1e-8
bissecao(f, precisao, a, b)

3.141592651605606

In [39]:
# Exemplo 2
f(x) = (x^3)-1
# Sabemos que a raíz nesse ponto é 1, pois (1^3)-1 = 0
b = -5
a = 5
precisao = 1e-16
bissecao(f, precisao, a, b)

0.9999999999999964

In [4]:
# Exemplo 3
f(x) = (ℯ^x)-1
b = -10
a = 10
precisao = 1e-10
# Sabemos que e^0-1 = 0
bissecao(f, precisao, a, b)

0.0

### Função Ponto Fixo

In [46]:
"""
RESUMO: É uma função que encontra raíz e consiste em encontrar uma função g : [a, b] → R tal que g(x) = x e com isso, construir uma sequência 
        xn de pontos da forma xn = g(xn−1), para um dada ponto inicial. Seja f : [a, b] → R contínua com f(a) ≤ a 
        e f(b) ≥ b. Existe ao menos um ponto c ∈ [a, b] tal que f(c) = c.
ENTRADA: funcao, precisao, chute_inicial
SAÍDA: raiz
"""

function ponto_fixo(g, precisao, x;
                  max_time = 10.0, max_iter = 1000,
                  )
    
    gx = g(x)
    
    # tolerância do valor da funcao
    ϵ = precisao * abs(gx)
    
    # define variaveis de tempo e qtd de iteraçoes
    iter = 0
    t0 = time()
    Δt = time() - t0
    
    # caso o algoritmo nao termine de nenhuma maneira esperada
    exitflag = :desconhecido
    
    # condicao para o algoritmo ter resolvido o problema
    resolvido = (abs(gx - x) ≤ ϵ)
    
    # condicao para o algoritmo pedir arrego
    cansado = (iter ≥ max_iter || Δt ≥ max_time)
    
    while !(resolvido || cansado)
        # calcula os novos valores de x e g(x)
        x = g(x) 
        gx = g(x)
        
        # calcula condicoes para o while
        iter = iter + 1
        Δt = time() - t0
        resolvido = (abs(gx - x) ≤ ϵ)
        cansado = (iter ≥ max_iter || Δt ≥ max_time)
    end
    
    
    if resolvido
        return x
    elseif cansado
        if iter ≥ max_iter
            return "ERRO: Chegamos ao número máximo de iterações"
        else
            return "ERRO: Chegamos ao máximo de tempo de execução"
        end
    end
    
    return x
end

ponto_fixo (generic function with 1 method)

#### Exemplos:

In [47]:
# Exemplo 1
fg(x) =  sin(x)/π
x = 2
ponto_fixo(fg, 1e-8, x)

3.1652829335191502e-9

In [48]:
# Exemplo 2
fg(x) =  x^3+x^2
x = 2
ponto_fixo(fg, 1e-8, x)

0

In [49]:
# Exemplo 3
fg(x) = 3x^5-7x^3
x = 10
ponto_fixo(fg, 1e-9, x)

0

### Função Newton

In [50]:
"""
RESUMO: Tem-se o objetivo de encontrar uma raíz para a função para isso, escolhe-se uma aproximação inicial 
        para esta. Após isso, calcula-se a equação da reta tangente (por meio da derivada) da função nesse ponto 
        e a interseção dela com o eixo das abcissas, a fim de encontrar uma melhor aproximação para a raiz. 
        Repetindo-se o processo, cria-se um método iterativo para encontrarmos a raiz da função. 
ENTRADA: funcao,derivada, precisao, chute_inicial
SAÍDA: raiz
"""

function newton(funcao,derivada, precisao, chute_inicial;
        max_tempo = 10.0, max_iter = 1000)
    x= chute_inicial
    fx = funcao(x) 
    ϵ  = precisao *abs(fx) #Calculando a ϵ com base na precisão dada
    #Inicialização de variáveis
    iter =0
    t0 = time()
    Δt = time() - t0
    
    resolvido = (abs(fx) ≤ ϵ) # Quando tivermos um valor de fx próximo o suficiente de 0, com base no ϵ
    cansado = (iter ≥ max_iter || Δt ≥ max_tempo) 
    # Aqui é onde temos o cálculo 
    while !(resolvido || cansado)
        fdx = derivada(x) # Calculando a derivada 
        if abs(fdx) ≤ ϵ
            return "ERRO: DERIVADA NULA"
        end
          
        x = x-fx/fdx # Calculando a equação do método de newton
        fx = f(x)
        iter +=1
        Δt = time()-t0
        
        resolvido = (abs(fx) ≤ ϵ)
        cansado = (iter ≥ max_iter || Δt ≥ max_tempo)
    end
    if resolvido
        return x
    elseif cansado
        if iter ≥ max_iter
            return "ERRO: Chegamos ao número máximo de iterações"
        else
            return "ERRO: Chegamos ao máximo de tempo de execução"
        end
    end
end

newton (generic function with 1 method)

#### Exemplos

In [51]:
# Exemplo 1
f(x) = x^5+x^2
fd(x) = 5x^4+2x
x = -15
newton(f,fd, 1e-8, x)

-1.0002056482183954

In [52]:
# Exemplo 2
f(x) = sin(x)
fd(x) = cos(x)
x = 2
newton(f,fd, 1e-8, x)

3.1415926536808043

In [53]:
# Exemplo 3
f(x) = 3x^5-7x^3+2
fd(x) = 15x^4-21x^2
x = 0.2
newton(f,fd, 1e-8, x)

1.4549501501230384

### Resolver PVI - Método de Euler

In [54]:
"""
RESUMO: O método de Euler consiste em resolver uma equação diferencial ordinária. Á ideia é encontrar uma solução 
        numérica para a equação diferencial no intervalo entre x_inicial e x_f .
        Primeiro, o intervalo é discretizado em n + 1 pontos:
            x_0 , x_1 , x_2 , x_3 …, x_n
        Que são obtidos assim:
            x_i = x + ih
        Onde h é a largura ou o tom dos subintervalos:
            (x_final-x_inicial)/numero_iterações
        Com a condição inicial, também é possível conhecer a derivada no início:
            y ‘(x_0) = f (x_0, y_0)
        Essa derivada representa a inclinação da reta tangente à curva da função y (x) exatamente no ponto:
            Ao = (x o , y o )
        Em seguida, uma previsão aproximada do valor da função y (x) é feita no seguinte ponto:
            y (x_1)≈ y_1
            y_1 = y_0 + (x_1 – x_0) f (x_0, y_0) = y_0 + hf (x_0, y_0)
        O próximo ponto aproximado da solução que corresponderia a:
            A_1 = (x_1, y_1)
        O procedimento é repetido para obter os pontos sucessivos
            A_2 , A_3 …, x_n
ENTRADA: derivada, x_inicial, y_inicial, x_final, n
SAÍDA: x,y
"""

function resolver_PVI(derivada,x_inicial,y_inicial,x_final,n)
    h =(x_final-x_inicial)/n
    y = y_inicial
    x = x_inicial
    for i = 1:n
        y = y + h * derivada(x, y)
        x = x + h
    end
    return x, y
end

resolver_PVI (generic function with 1 method)

#### Exemplos:

In [55]:
# Exemplo 1
derivada(x,y) = (x - y)/x
x_inicial = 2
y_inicial = 2
x_final = 3
n = 10
resolver_PVI(derivada,x_inicial,y_inicial,x_final,n)

(3.000000000000001, 2.155172413793104)

In [56]:
# Exemplo 2
derivada(x,y) = (x^2 - y)/x
x_inicial = 2
y_inicial = 2
x_final = 3
n = 10
resolver_PVI(derivada,x_inicial,y_inicial,x_final,n)

(3.000000000000001, 3.408620689655173)

In [57]:
# Exemplo 3
derivada(x,y) = cos(x)-sin(y)
x_inicial = 2
y_inicial = 2
x_final = 3
n = 10
resolver_PVI(derivada,x_inicial,y_inicial,x_final,n)

(3.000000000000001, 0.4018171518643542)

### Resolver PVC

In [96]:
function resolver_PVC(segunda_derivada, x_inicial, y_inicial, x_final, y_final, n)  # Não dá para ser feita genericamente
     A,b = # Usuário monta um sistema linear usando diferenças finitas(segunda_derivada(x,y), x_inicial, y_inicial, x_final, y_final,n)
     y = resolver_um_sistema(A,b)
return y
end

resolver_PVC (generic function with 1 method)

### Resolve Diagonal

In [96]:
# Resolve diagonal
""""
RESUMO: Resolve um sistema linear Ax=b que tenha uma matriz diagonal
ENTRADA: Matriz A, vetor b
SAIDA: vetor x (solucao do sistema)
"""

function resolve_diagonal(A, b)
    rows, cols = size(A)
    
    # verifica se a matriz é quadrada
    if rows != cols
        error("A matriz deve ser quadrada!")
    end
    
    # Inicializa o vetor solucao
    x = zeros(rows)
    
    # Calcula cada elemento do vetor x
    # Dividindo o valor de cada linha de b pelos valores na diagonal de A
    for i = 1:rows
        x[i] = b[i] / A[i,i]
    end
    
    return x
end

resolve_diagonal (generic function with 1 method)

#### Exemplo 1 (n = 3)

In [112]:
A = [2  0   0;
     0  7   0;
     0  0 -11]

b = [-6, 7, 110]

x = resolve_diagonal(A, b)

norm(A*x - b)

0.0

#### Exemplo 2 (n = 5)

In [113]:
n = 5
A = Diagonal(rand(5,5))

b = rand(5)

x = resolve_diagonal(A, b)

norm(A * x - b)

0.0

#### Exemplo 3 (n = 20)

In [114]:
n = 20
A = Diagonal(rand(n, n))
b = rand(n)
x = resolve_diagonal(A, b)
norm(A*x - b)

5.721958498152797e-17

### Resolve Triangular Superior

In [31]:
# Resolve triangular superior
""""
RESUMO: Resolve um sistema linear Ax=b que tenha uma matriz triangular superior.
        Ou seja, todos os elementos abaixo da diagonal estao zerados.
ENTRADA: Matriz A, vetor b
SAIDA: vetor x (solucao do sistema)
"""

function resolve_triangular_superior(A, b)
    rows, cols = size(A)
    
    # verifica se a matriz é quadrada
    if rows != cols
        error("A matriz deve ser quadrada!")
    end
    
    # Inicializa o vetor solucao
    x = zeros(rows)
    
    # Calcula cada elemento do vetor x
    # substituindo em cada linha, de baixo para cima
    for i = rows:-1:1
        x[i] = b[i]  # pegamos o bᵢ
        for j = i+1:cols
            x[i] -= A[i, j] * x[j]  # subtraimos de bᵢ todos os x's que ja 
                                    # foram calculados (multiplicados pelos coeficientes)
        end
        x[i] = x[i] / A[i, i]  # Dividimos o resultado pelo coeficiente de xᵢ
    end
    
    return x
end

resolve_triangular_superior (generic function with 1 method)

#### Exemplo 1 (n = 3)

In [124]:
A = [-2    7  19;
      0  0.5 -10;
      0    0   3]

3×3 Matrix{Float64}:
 -2.0  7.0   19.0
  0.0  0.5  -10.0
  0.0  0.0    3.0

In [125]:
b = [2, 8.5, -11]

3-element Vector{Float64}:
   2.0
   8.5
 -11.0

In [126]:
x = resolve_triangular_superior(A, b)

3-element Vector{Float64}:
 -233.0
  -56.33333333333333
   -3.6666666666666665

In [127]:
norm(A*x - b)

2.842170943040401e-14

#### Exemplo 2 (n = 5)

In [130]:
n = 5
A = UpperTriangular(rand(n,n))
b = rand(n)
x = resolve_triangular_superior(A, b)
norm(A*x - b)

1.2412670766236366e-16

#### Exemplo 3 (n = 20)

In [137]:
n = 20
A = UpperTriangular(rand(n,n))
b = rand(n)
x = resolve_triangular_superior(A, b)
norm(A*x - b)

7.405661942207508e-14

### Resolve Triangular Inferior

In [32]:
# Resolve triangular inferior
""""
RESUMO: Resolve um sistema linear Ax=b que tenha uma matriz triangular inferior.
        Ou seja, todos os elementos acima da diagonal estao zerados.
ENTRADA: Matriz A, vetor b
SAIDA: vetor x (solucao do sistema)
"""

function resolve_triangular_inferior(A, b)
    rows, cols = size(A)
    
    # verifica se a matriz é quadrada
    if rows != cols
        error("A matriz deve ser quadrada!")
    end
    
    # Inicializa o vetor solucao
    x = zeros(rows)
    
    # Calcula cada elemento do vetor x
    # substituindo em cada linha, de cima para baixo
    for i = 1:rows
        x[i] = b[i]  # pegamos o bᵢ
        for j = i-1:-1:1
            x[i] -= A[i, j] * x[j]  # subtraimos de bᵢ todos os x's que ja 
                                    # foram calculados (multiplicados pelos coeficientes)
        end
        x[i] = x[i] / A[i, i]  # Dividimos o resultado pelo coeficiente de xᵢ
    end
    
    return x
end

resolve_triangular_inferior (generic function with 1 method)

#### Exemplo 1 (n = 3)

In [140]:
A = [  9    0   0;
      -7    1   0;
    56.3   85  -2]

3×3 Matrix{Float64}:
  9.0   0.0   0.0
 -7.0   1.0   0.0
 56.3  85.0  -2.0

In [141]:
b = [30, 2.75, -4.1]

3-element Vector{Float64}:
 30.0
  2.75
 -4.1

In [142]:
x = resolve_triangular_inferior(A, b)

3-element Vector{Float64}:
    3.3333333333333335
   26.083333333333336
 1204.425

In [143]:
norm(A*x - b)

9.059419880941277e-14

#### Exemplo 2 (n = 5)

In [147]:
n = 5
A = LowerTriangular(rand(n,n))
b = rand(n)
x = resolve_triangular_inferior(A, b)
norm(A*x - b)

1.4936523181711914e-15

#### Exemplo 3 (n = 20)

In [150]:
n = 20
A = LowerTriangular(rand(n,n))
b = rand(n)
x = resolve_triangular_inferior(A, b)
norm(A*x - b)

2.2978454670130187e-13

### Eliminação Gaussiana

In [29]:
# Eliminacao gaussiana
"""
RESUMO: Funçao que executa a eliminacao gaussiana em uma matriz A e um vetor b.
ENTRADA: Matriz A, vetor b
SAIDA: Nada
OBS: Modifica as entradas!
"""

function eliminacao_gaussiana(A, b)
    rows, cols = size(A)
    for j = 1:cols
        non_zero_cols = j+1:cols
        for i = j+1:rows
            mᵢⱼ = A[i, j] / A[j, j]  # multiplicador que vai zerar a coluna
            A[i, non_zero_cols] -= mᵢⱼ * A[j, non_zero_cols]  # subtracao das linhas de A
            A[i, j] = 0  # zera cada elemento abaixo do pivô na coluna j
            b[i] -= mᵢⱼ * b[j]  # subtracao das linhas de b
        end
    end
end

eliminacao_gaussiana (generic function with 1 method)

#### Exemplo 1 (n = 3)

In [3]:
n = 3
A = rand(n,n)
Acopy = copy(A)

3×3 Matrix{Float64}:
 0.202406  0.673815  0.880767
 0.587993  0.708312  0.0878447
 0.871601  0.163057  0.278384

In [4]:
b = rand(n)
bcopy = copy(b)

3-element Vector{Float64}:
 0.8128419958346671
 0.18778020257382844
 0.29418940502129476

In [5]:
eliminacao_gaussiana(A, b)
A

3×3 Matrix{Float64}:
 0.202406   0.673815   0.880767
 0.0       -1.24913   -2.4708
 0.0        0.0        1.90247

In [6]:
b

3-element Vector{Float64}:
  0.8128419958346671
 -2.173538677426604
  1.5590775743480592

In [11]:
# Testar (resolvendo o sistema)
x = resolve_triangular_superior(A, b)
norm(Acopy * x - bcopy)

2.9893669801409083e-16

#### Exemplo 2 (n = 5, x random)

In [12]:
n = 5

5

In [13]:
A = rand(n,n)
Acopy = copy(A)

5×5 Matrix{Float64}:
 0.447115   0.526316  0.438287  0.978176  0.454452
 0.601031   0.652873  0.988625  0.35596   0.373821
 0.68519    0.700868  0.463585  0.643119  0.422305
 0.890906   0.479626  0.684703  0.453157  0.0877369
 0.0572552  0.548967  0.323097  0.759533  0.897974

In [14]:
b = rand(n)
bcopy = copy(b)

5-element Vector{Float64}:
 0.5640261287581383
 0.8292528488448687
 0.6969664395383763
 0.7414681127930645
 0.44060932503869843

In [15]:
eliminacao_gaussiana(A, b)
A

5×5 Matrix{Float64}:
 0.447115   0.526316    0.438287   0.978176   0.454452
 0.0       -0.0546233   0.399461  -0.958946  -0.237073
 0.0        0.0        -0.981017   0.999619   0.184598
 0.0        0.0         0.0        4.06196    0.833536
 0.0        0.0         0.0        0.0        0.275119

In [16]:
b

5-element Vector{Float64}:
  0.5640261287581383
  0.07106466711198633
 -0.30489362439098844
  0.22929438383551992
  0.04090904190613345

In [17]:
# Testar (resolvendo o sistema)
x = resolve_triangular_superior(A, b)
norm(Acopy * x - bcopy)

1.1102230246251565e-16

#### Exemplo 3 (n = 20, x random)

In [18]:
n = 20
A = rand(n,n)
Acopy = copy(A)
b = rand(n)
bcopy = copy(b)
eliminacao_gaussiana(A, b)

# Testar (resolvendo o sistema)
x = resolve_triangular_superior(A, b)
norm(Acopy * x - bcopy)

8.897729340113409e-15

### Resolve Cheia

In [27]:
# Resolve cheia
"""
RESUMO: Resolve um sistema linear com uma matriz quadrada "cheia".
        Ou seja, que nao é triangular nem diagonal.
        Primeiro aplicamos a eliminacao gaussiana e depois resolvemos o sistema 
        com a matriz triangular superior.
ENTRADA: Matriz A, vetor b
SAIDA: Vetor x (solucao do sistema)
OBS: Modifica as entradas!
"""

function resolve_cheia(A, b)
    eliminacao_gaussiana(A,b)
    x = resolve_triangular_superior(A,b)
    return x
end

resolve_cheia (generic function with 1 method)

#### Exemplo 1 (n = 3)

In [33]:
A = [2   -4      1;
     9   18  -7.34;
     0  0.2     45]
Acopy = copy(A)

3×3 Matrix{Float64}:
 2.0  -4.0   1.0
 9.0  18.0  -7.34
 0.0   0.2  45.0

In [34]:
b = [-1, 84, 4.333]
bcopy = copy(b)

3-element Vector{Float64}:
 -1.0
 84.0
  4.333

In [35]:
x = resolve_cheia(A, b)

3-element Vector{Float64}:
 4.43011538689126
 2.486367285350796
 0.08523836762066314

In [36]:
norm(Acopy * x - bcopy)

0.0

#### Exemplo 2 (n = 5)

In [37]:
n = 5
A = rand(n,n)
Acopy = copy(A)
b = rand(n)
bcopy = copy(b)
x = resolve_cheia(A, b)
norm(Acopy * x - bcopy)

6.568167990716596e-16

#### Exemplo 3 (n = 20)

In [20]:
n = 20
A = rand(n,n)
Acopy = copy(A)
b = rand(n)
bcopy = copy(b)
x = resolve_cheia(A, b)
norm(Acopy * x - bcopy)

1.9303918037323056e-14

### Decomposição LU 

In [81]:
# Decomposicao LU
"""
RESUMO: Aplica a eliminacao gaussiana na matriz e salva os multiplicadores utilizados
        para decompor uma matriz A em uma matriz triangular inferior L(Lower) e uma matriz
        triangular superior U(Upper). A matriz L pode ser utilizada para aplicar a eliminacao
        gaussiana em um vetor com complexidade O(n^2) em vez de O(n^3).
ENTRADA: Matriz A
SAIDA: Matriz L e matriz U
"""
function decomposicao_LU(A)
    rows, cols = size(A)
    U = copy(A)
    # Inicia L como uma matriz identidade
    L = Matrix(1.0I, rows, cols)
    for j = 1:cols
        non_zero_cols = j+1:cols
        for i = j+1:rows
            L[i, j] = U[i, j] / U[j, j]  # multiplicador que vai zerar a coluna
            U[i, non_zero_cols] -= L[i, j] * U[j, non_zero_cols]  # subtracao das linhas de A
            U[i, j] = 0  # zera cada elemento abaixo do pivô na coluna j
        end
    end
    return L, U
end

decomposicao_LU

#### Exemplo 1 (n = 3)

In [90]:
A = [45  6.777  -1.59;
      3      9   -0.1;
     10     45      1]

3×3 Matrix{Float64}:
 45.0   6.777  -1.59
  3.0   9.0    -0.1
 10.0  45.0     1.0

In [91]:
b = [9.0, 18.0, -75.1]

3-element Vector{Float64}:
   9.0
  18.0
 -75.1

In [92]:
L, U = decomposicao_LU(A)
L

3×3 Matrix{Float64}:
 1.0        0.0      0.0
 0.0666667  1.0      0.0
 0.222222   5.08809  1.0

In [93]:
U

3×3 Matrix{Float64}:
 45.0  6.777   -1.59
  0.0  8.5482   0.006
  0.0  0.0      1.3228

In [100]:
A - L * U

3×3 Matrix{Float64}:
 0.0   0.0          0.0
 0.0   0.0          0.0
 0.0  -7.10543e-15  0.0

#### Exemplo 2 (n = 5)

In [101]:
n = 5
A = rand(n,n)
b = rand(n)
L, U = decomposicao_LU(A)
A - L*U

5×5 Matrix{Float64}:
  0.0           0.0           0.0           0.0           0.0
  0.0           0.0           0.0           0.0           0.0
  0.0          -1.11022e-16  -1.11022e-16   0.0           0.0
  0.0           0.0           0.0          -2.77556e-17  -5.55112e-17
 -1.11022e-16   0.0          -5.55112e-17  -5.55112e-17  -5.77316e-15

#### Exemplo 3 (n = 20)

In [102]:
n = 20
A = rand(n,n)
b = rand(n)
L, U = decomposicao_LU(A)
A - L*U

20×20 Matrix{Float64}:
  0.0           0.0           0.0          …   0.0           0.0
 -1.11022e-16   0.0           0.0             -2.22045e-16   0.0
  0.0          -1.38778e-17   0.0             -2.77556e-17   1.38778e-17
  0.0           0.0           0.0              2.22045e-16   0.0
  0.0           0.0           0.0             -6.66134e-16   0.0
  0.0           0.0           0.0          …   0.0           0.0
  0.0           0.0          -1.11022e-16     -3.33067e-16  -1.11022e-16
  0.0           2.77556e-17   0.0             -7.77156e-16   3.33067e-16
  0.0           0.0           0.0              6.66134e-16  -1.04083e-16
  0.0           0.0           0.0             -4.44089e-16   2.22045e-16
  0.0           0.0           0.0          …  -1.11022e-16   0.0
  0.0           0.0           1.11022e-16     -6.66134e-16  -5.55112e-16
  0.0          -2.77556e-17   0.0              0.0          -8.32667e-17
  0.0          -5.55112e-17   0.0              2.22045e-16   6.66134e-16
  0

### Resolve Pós-LU 

In [104]:
# resolve pos-LU
"""
RESUMO: Esta funcao deve ser utilizada depois que fizemos uma decomposicao LU.
        Quando fazemos essa decomposicao LU em um sistema linear temos o seguinte:

        Ax = b
        LUx = b; y = Ux
        Ly = b
        y = L⁻¹b
        Ux = L⁻¹b
        x = U⁻¹L⁻¹b

        Esta funcao primeiro vai resolver o sistema Ly = b para depois resolver 
        o sistema Ux = y.
        A complexidade dessa funcao é apenas O(n²), contra O(n³) de uma eliminacao 
        gaussiana normal.

ENTRADA: Matriz L, Matriz U, Vetor b

SAIDA: Vetor x (solucao de LUx = b)
"""
function resolve_pos_LU(L, U, b)
    y = resolve_triangular_inferior(L, b)
    x = resolve_triangular_superior(U, y)
    return x
end

resolve_pos_LU

#### Exemplo 1 (n = 3)

In [105]:
L = [  1     0  0;
      23     1  0;
     -91  1.11  1]

3×3 Matrix{Float64}:
   1.0  0.0   0.0
  23.0  1.0   0.0
 -91.0  1.11  1.0

In [106]:
U = [6  70     -10
     0 241  99.999
     0   0      -1]

3×3 Matrix{Float64}:
 6.0   70.0  -10.0
 0.0  241.0   99.999
 0.0    0.0   -1.0

In [107]:
b = [64, 13.171, -0.4]

3-element Vector{Float64}:
 64.0
 13.171
 -0.4

In [110]:
x = resolve_pos_LU(L, U, b)

3-element Vector{Float64}:
 -48353.82279037807
   3082.2562120324064
  -7442.90019

In [112]:
norm(L * U * x - b)

3.326949296948146e-9

#### Exemplo 2 (n = 5)

In [115]:
n = 5
L = UnitLowerTriangular(rand(n,n))
U = UpperTriangular(rand(n,n))
b = rand(n)
x = resolve_pos_LU(L, U, b)
norm(L*U*x - b)

0.0

#### Exemplo 3 (n = 20)

In [119]:
n = 20
L = UnitLowerTriangular(rand(n,n))
U = UpperTriangular(rand(n,n))
b = rand(n)
x = resolve_pos_LU(L, U, b)
norm(L*U*x - b)

8.345230562421201e-12

### Acha inversa 

In [208]:
# Acha inversa
"""
RESUMO: Vamos achar a inversa de uma matriz A utilizando a decomposicao LU.
        Primeiro, vamos decompor A em L * U.
        Depois, vamos usar os vetores canonicos da matriz identidade como
        valores de b no sistema linear LUx = b.
        Ou seja, vamos descobrir os vetores que fazem parte da matriz inversa de A
        calculando um x para cada vetor na matriz identidade com as mesmas dimensoes que A.

ENTRADA: Matriz A

SAIDA: Matriz A⁻¹ (inversa de A)
"""
function acha_inversa(A)
    rows, cols = size(A)  # Pega as dimensoes de A
    L, U = decomposicao_LU(A)  # Faz a decomposicao LU
    A⁻¹ = []
    for i = 1:cols
        id_vector = zeros(cols)
        id_vector[i] = 1.0  # Monta cada coluna da identidade
        x = resolve_pos_LU(L, U, id_vector)  # resolve um sistema para cada coluna da identidade
        append!(A⁻¹, x)  # Adiciona os elemetos do vetor x numa array
    end
    A⁻¹ = reshape(A⁻¹, rows, cols)  # Formata a array como uma matriz quadrada
    return A⁻¹
end

acha_inversa

#### Exemplo 1 (n = 3)

In [209]:
A = [  2  49     -12;
     0.5  10       1;
      -9   0  54.888]

3×3 Matrix{Float64}:
  2.0  49.0  -12.0
  0.5  10.0    1.0
 -9.0   0.0   54.888

In [210]:
A⁻¹ = acha_inversa(A)

3×3 Matrix{Any}:
 -0.310453    1.52122     -0.0955885
  0.0206132  -0.00100453   0.0045249
 -0.0509051   0.249435     0.00254525

In [211]:
A * A⁻¹

3×3 Matrix{Any}:
  1.0          0.0          6.93889e-18
  2.08167e-17  1.0          8.67362e-18
 -2.66454e-15  1.06581e-14  1.0

#### Exemplo 2 (n = 5)

In [212]:
n = 5
A = rand(n,n)
b = rand(n)
A⁻¹ = acha_inversa(A)
A * A⁻¹ ≈ I

true

#### Exemplo 3 (n = 20)

In [213]:
n = 20
A = rand(n,n)
b = rand(n)
A⁻¹ = acha_inversa(A)
A * A⁻¹ ≈ I

true

### Interpolação Coeficientes

In [23]:
"""
RESUMO: É um método no qual instanciamos um novo conjunto de dados utilizando interpolação polinomial em 
        vista de construir novos pontos de dados no alcance de pontos já conhecidos.
ENTRADA: pontos_x, pontos_y, grau
SAIDA: coeficientes
"""
function interpolação_coeficientes(pontos_x,pontos_y, grau)
    V=matrix_de_vandermonde(pontos_x,grau)
    coeficientes=resolve_cheia(V,pontos_y)
    return coeficientes
end

interpolação_coeficientes

#### Exemplos

In [33]:
#Exemplo 1
x = [1000.0, 300.0, 6000.0]
y = [4.0,6.0, 11.0]
cs= interpolação_coeficientes(x,y,3)
print("c₀ = $(cs[1]), c₁ = $(cs[2]), c₂ = $(cs[3])")

c₀ = 7.081203007518797, c₁ = -0.003828070175438596, c₂ = 7.468671679197994e-7

In [76]:
#Exemplo 2
x = [50.0, 75.0, 45.0, 60.0, 44.0, 5.0]
y = [1.0, 7.0, 8.0, 10.0, 15.0, 20.0]
cs= interpolação_coeficientes(x,y,length(x))
print("c₀ = $(cs[1]), c₁ = $(cs[2]), c₂ = $(cs[3]), c₃ = $(cs[6]), c₄ = $(cs[5]), c₅ = $(cs[6])")

c₀ = -2062.775522864238, c₁ = 564.4589067652769, c₂ = -33.47130855613919, c₃ = 3.847534783018654e-5, c₄ = -0.009181851040238139, c₅ = 3.847534783018654e-5

In [77]:
#Exemplo 3
x = [11.0, 250.0,145.0]
y = [4.0,15.0, 27.0]
cs= interpolação_coeficientes(x,y,3)
print("c₀ = $(cs[1]), c₁ = $(cs[2]), c₂ = $(cs[3])")

c₀ = 0.20376301397971253, c₁ = 0.3582722966161423, c₂ = -0.0011963493946882445

### Função Matriz de Vandermonde
É uma matriz que tem um formato:

$M=\left(\begin{array}{llll} 1 & 1 & \cdots & 1 \\ a_1 & a_2 & \cdots & a_n \\ a_1^2 & a_2^2 & \cdots & a_n^2 \\ \vdots & \vdots & \ddots & \vdots \\ a_1^{n-1} & a_2^{n-1} & \cdots & a_n^{n-1} \end{array}\right)$

In [25]:
"""
Entrada: a função recebe os pontos (x, y) e o grau da função.
Saída: matriz de Vandermonde no formato citado acima.
"""

function matrix_de_vandermonde(pontos_x,grau)
    # Aqui criamos a matriz zerada com as dimensões certas
    matrix_de_vandermonde = zeros(length(x), grau)
    
    # O loop abaixo preenche a matriz de vandermonde
    for linha = 1:length(x)
        for coluna = 1:grau
            matrix_de_vandermonde[linha, coluna] = x[linha]^(coluna - 1)
        end
    end
    return matrix_de_vandermonde
end

matrix_de_vandermonde (generic function with 1 method)

### Função Lagrange

In [34]:
"""
RESUMO: Recebe uma série de pontos (x,y) e um ponto especifico xp.
        Descobre uma funcao que interpola os pontos (x,y) e descobre a imagem
        de xp nessa funcao.
        O grau do polinomio vai ser definido pelo numero de pontos passados.

ENTRADA: Vetor pontos_x, Vetor pontos_y, Float xp

SAIDA: Float yp (imagem de xp na funcao que interpola os pontos (x,y))
"""
function lagrange(x, y, xp)
    # pega o numero de pontos
    n = length(x)
    # resultado da interpolação
    yp = 0
    # Implementando lagrange
    for i in 1:n
        p = 1
        for j in 1:n
            if i != j
                p *= (xp - x[j])/(x[i] - x[j])
            end
        end
        yp += p * y[i]
    end
    return yp
end

lagrange

#### Exemplos

In [35]:
#Exemplo 1
x = [1.0, 8.0, 10.0, 15.0]
y = [15.0, 25.0, 1.0, 12.0]
for xi in x
    println(lagrange(x, y, xi))
end
println(lagrange(x, y, 7.))

15.0
25.0
1.0
12.0
37.050340136054416


In [36]:
#Exemplo 2
x = [1., 2., 7.]
y = [2., 5., 4.]
for xi in x
    println(lagrange(x, y, xi))
end
println(lagrange(x, y, 4.))

2.0
5.0
4.0
7.8


In [37]:
#Exemplo 3
n = 10
x = rand(10)
y = rand(10)
for i = 1:n
    println("Valor real: $(y[i]) / Valor por lagrange: $(lagrange(x, y, x[i]))")
end
println(lagrange(x, y, 1.))

Valor real: 0.4546981494338933 / Valor por lagrange: 0.4546981494338933
Valor real: 0.0684998732401958 / Valor por lagrange: 0.0684998732401958
Valor real: 0.6789581074321636 / Valor por lagrange: 0.6789581074321636
Valor real: 0.6842519572680417 / Valor por lagrange: 0.6842519572680417
Valor real: 0.6605895651295604 / Valor por lagrange: 0.6605895651295604
Valor real: 0.23748661174261776 / Valor por lagrange: 0.23748661174261776
Valor real: 0.12787285601721865 / Valor por lagrange: 0.12787285601721865
Valor real: 0.11200564945256097 / Valor por lagrange: 0.11200564945256097
Valor real: 0.427979027177944 / Valor por lagrange: 0.427979027177944
Valor real: 0.5494217349800894 / Valor por lagrange: 0.5494217349800894
-2109.9951378948545


### Função Aplica

In [5]:
"""
Objetivo da função: A função "Aplica" transforma uma lista de coeficientes em uma função. 
                    Ou seja, a partir de uma lista de coeficientes e um valor para x, a função 
                    retornar o valor em x para a função que conhecemos apenas os coeficientes.
Entrada: coeficientes de uma função e um valor de x.
Saída: valor da função desconhecida no ponto x.
"""

function aplica(coeficientes, valor_de_x, grau = length(coeficientes)-1)
    len = grau + 1
    accx = 1
    acc = 0
    for i = 1:len
        acc += coeficientes[i] * accx
        accx *= valor_de_x
    end
    return acc
end

aplica (generic function with 2 methods)

#### Exemplos:

In [92]:
# Exemplo 1
# Imagine que desconhecemos a função 2(x^2)+3(x)-5 = 0 ; cujas raízes são [1, -5/2]. Desconfio 
# que 1 seja raiz e quero verificar

print(aplica([-5,3,2],1), "\n")

# Desconfio que -5/2 seja raiz e quero verificar
print(aplica([-5,3,2],-5/2), "\n")

#Quero verificar o valor da função no ponto x = 5
print(aplica([-5,3,2],5), "\n")

0
0.0
60


In [93]:
# Exemplo 2
# Quero calcular o valor da função (x^3)-574 = 0 ; no ponto x = 10

aplica([-574,0,0,1],10)

426

In [94]:
# Exemplo 3
# Imagine que quero descobrir uma raiz da função (x^3)+2x-12 = 0 
# E sei que existe uma raiz entre x = -1000 e x = 1000

raiz = -999
for i = -1000:1000
    y = aplica([-12,2,0,1],i)
    if y == 0
        raiz = i
        break
    end
end
return(raiz)

2

### Função Regressão Coeficientes Lineares

In [138]:
"""
Objetivo da função: Encontrar o valor dos coeficientes, de uma função aproximada à função 
                    desejada, que torna mínima a soma dos quadrados das diferenças entre a 
                    função desejada e sua aproximação.
Entrada: pontos (x,y) e o grau da funçã desejada.
Saída: coeficientes
"""

function regressao_coeficientes_lineares(pontos_x,pontos_y, grau)
    V = matrix_de_vandermonde(pontos_x,grau)  # matrix alta (com muitas linhas e poucas colunas) 
    Vᵀ = transpose(V)  # VᵀVc = Vᵀy
    coeficientes = Vᵀ * V \ Vᵀ * y  # Coeficientes são calculados atráves de mínimos quadrados
    return coeficientes
end

regressao_coeficientes_lineares (generic function with 1 method)

#### Exemplos:

In [139]:
# Exemplo 1
# Temos o mesmo exemplo que fizemos anteriormente utilizando interpolação de coeficientes
x = [1000.0, 300.0, 6000.0]
y = [4.0,6.0, 11.0]
coef= regressao_coeficientes_lineares(x,y,3)
print("c₀ = $(coef[1]) 
     \nc₁ = $(coef[2]) 
     \nc₂ = $(coef[3])")

c₀ = 7.081203007518854 
     
c₁ = -0.0038280701754386916 
     
c₂ = 7.468671679198137e-7

In [140]:
# Exemplo 2
# Temos o mesmo exemplo que fizemos anteriormente utilizando interpolação de coeficientes
x = [50.0, 75.0, 45.0, 60.0, 44.0, 5.0]
y = [1.0, 7.0, 8.0, 10.0, 15.0, 20.0]
coef= regressao_coeficientes_lineares(x,y,length(x))
print("c₀ = $(coef[1])
     \nc₁ = $(coef[2])
     \nc₂ = $(coef[3])
     \nc₃ = $(coef[6])
     \nc₄ = $(coef[5])
     \nc₅ = $(coef[6])")

c₀ = -2062.77113627787
     
c₁ = 564.4577108950482
     
c₂ = -33.47123627642948
     
c₃ = 3.847525958436212e-5
     
c₄ = -0.009181830376718415
     
c₅ = 3.847525958436212e-5

In [141]:
# Exemplo 3
# Temos o mesmo exemplo que fizemos anteriormente utilizando interpolação de coeficientes
x = [11.0, 250.0,145.0]
y = [4.0,15.0, 27.0]
coef = regressao_coeficientes_lineares(x,y,3)
print("c₀ = $(coef[1]) 
     \nc₁ = $(coef[2]) 
     \nc₂ = $(coef[3])")

c₀ = 0.20376301397977503 
     
c₁ = 0.3582722966161403 
     
c₂ = -0.0011963493946882372

### Função Mínimos Quadrados

In [43]:
"""
Objetivo da função: Encontrar o valor dos coeficientes, de uma função aproximada à função 
                    desejada, que torna mínima a soma dos quadrados das diferenças entre a 
                    função desejada e sua aproximação.
Entrada: pontos (x,y) e o grau do polinômio desejado.
Saída: coeficientes da função de grau desejado que melhor aproximam a função cujos pontos 
       foram passados.
"""

function resolver_um_sistema_por_minimos_quadrados(x, y, grau)
    n_coeficientes = grau + 1
    V = zeros(length(x), n_coeficientes)  # Aqui criamos uma matriz zerada com as dimensões certas
    
    # No loop abaixo, preenchemos a matriz de vandermonde
    for linha = 1:length(x)
        for coluna = 1:n_coeficientes
            V[linha, coluna] = x[linha]^(coluna - 1)
        end
    end
    
    Vᵀ = transpose(V)  # VᵀVc = Vᵀy
    return Vᵀ * V \ Vᵀ * y
end

resolver_um_sistema_por_minimos_quadrados (generic function with 1 method)

#### Exemplos:

In [150]:
# Exemplo 1
# Suponha que temos os seguintes pontos que descrevem a trajetória de um objeto com o passar do
# tempo. Queremos descobrir qual reta melhor descreve a distribuição dos pontos:
# (0,0), (5,10), (15,50), (25,63), (45, 73), (60,90), (156,468)

x = [0, 5, 15, 25, 45, 60, 156]
y = [0, 10, 50, 63, 73, 90, 468]
grau = 2
coeficientes = resolver_um_sistema_por_minimos_quadrados(x, y, grau)
aproxC₀ = round(coeficientes[1], digits = 2)
aproxC₁ = round(coeficientes[2], digits = 2)
aproxC₂ = round(coeficientes[3], digits = 2)
print("Melhor parábola que descreve a distribuição dos pontos citados: ",aproxC₂, "x^2 + ", aproxC₁, "x + ", aproxC₀, "\n")

Melhor parábola que descreve a distribuição dos pontos citados: 0.01x^2 + 0.67x + 15.89


In [152]:
# Exemplo 2
# Vamos supor que um objeto foi imerso em gelo e sua temperatura foi medida algumas vezes em
# intervalos de 2 em 2 min, conforme a "tabela" abaixo. Queremos saber uma boa aproximação para 
# a temperatura do objeto no minuto 6
#       minutos ->  1 | 3 | 5 | 7 | 9
#   temperatura -> 18 | 10| 7 | 5 | 4

min = [1, 3, 5, 7, 9]
temp = [18, 10, 7, 5, 4]
grau = 2
coeficientes = resolver_um_sistema_por_minimos_quadrados(min, temp, grau)
aproximacao = round(coeficientes[3]*36 + coeficientes[2]*6 + coeficientes[1], digits = 2)
print("Aproximação para a temperatura do objeto no minuto 6: $aproximacao ºC\n")

Aproximação para a temperatura do objeto no minuto 6: 5.27 ºC


In [151]:
# Exemplo 3
# Suponha que temos que encontrar a melhor função de grau 1 que se adapta aos pontos
# (2, 500), (4, 1500), (8, 5000)
x = [2, 4, 8]
y = [500, 1500, 5000]
grau = 1
coeficientes = resolver_um_sistema_por_minimos_quadrados(x, y, grau)
aproxC₀ = round(coeficientes[1], digits = 2)
aproxC₁ = round(coeficientes[2], digits = 2)
print("Melhor função de grau 1 que se adapta aos pontos citados: ", aproxC₀,"x + ", aproxC₁, "\n")

Melhor função de grau 1 que se adapta aos pontos citados: -1250.0x + 767.86


### Função Integração

In [99]:
"""
Objetivo da função: Encontrar o valor exato para a integral da função desejada.
Entrada: primitiva da função desejada, ponto a (ponto inicial -> x0), ponto b (ponto 
         final -> xn).
Saída: valor exato da integral de f entre os pontos [a,b].
"""

function integracao(F, a, b) 
    integralExata = F(b) - F(a)
    return integralExata
end

integracao (generic function with 1 method)

#### Exemplos:

In [16]:
# Exemplo 1
# Imagine que queremos determinar a área limitada pelo círculo
# (x^2) + (y^2) = 4 no primeiro quadrante

f(x) = sqrt(4-x^2)
F(x) = 2*asin(1/2*x)+sin(2asin(1/2*x)) # primitiva de f(x)
a, b = 0.0, 2.0
integral = integracao(F, a, b)
print("Integral exata = ", integral, "\n")

Integral exata = 3.141592653589793


In [17]:
# Exemplo 2
# Imagine que queremos determinar a área exata limitada pela função
# (e^x) entre os pontos x = 0 e x = 1

f(x) = exp(x)
F(x) = exp(x) # primitiva de f(x)
a, b = 0.0, 1.0
integral = integracao(F, a, b)
print("Integral exata = ", integral, "\n")

Integral exata = 1.718281828459045


In [19]:
# Exemplo 3
# Imagine que queremos determinar a área exata limitada pela função
# (x^3+5x-4) entre os pontos x = 0 e x = 5

f(x) = (x^3)+5x-4
F(x) = ((x^4)/4)+((5x^2)/2)-4x # primitiva de f(x)
a, b = 0.0, 5.0
integral = integracao(F, a, b)
print("Integral exata = ", integral, "\n")

Integral exata = 198.75


### Função Regra do Retângulo

In [177]:
"""
Objetivo: Encontrar uma aproximação para a integral da função desejada através de 
          retas ligando os n pontos que temos da função (formando retângulos entre 
          as retas e o eixo x e aproximando a integral final com as áreas desses 
          retângulos).
Entrada: função, ponto a (ponto inicial -> x0), ponto b (ponto final -> xn), n (número de 
         pontos que usaremos).
Saída: aproximação para o valor da integral de f entre os pontos [a,b].
"""

function regra_do_retangulo(f, a, b, n; T=Float64)
    h = (b - a) / n
    S = zero(T)
    for i = 0:n
        x = a + i * h
        S += f(x)
    end
    aproxIntegral = S*h   
    return aproxIntegral
end

regra_do_retangulo (generic function with 1 method)

#### Exemplo 1:

In [178]:
# Exemplo 1
# Imagine que queremos determinar uma aproximação para a área limitada pelo círculo
# (x^2) + (y^2) = 4 no primeiro quadrante, com 10 pontos

f(x) = sqrt(4-x^2)
a, b = 0.0, 2.0
pontos = 10
aproximacao = regra_do_retangulo(f, a, b, pontos)
print("Aproximação pela Regra do Retângulo = ", round(aproximacao, digits=4), "\n")

# Valor real = 3.14159 (aproximadamente)

Aproximação pela Regra do Retângulo = 3.3045


In [179]:
# Exemplo 2
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (e^x) entre os pontos x = 0 e x = 1 ; usando 20 pontos

f(x) = exp(x)
a, b = 0.0, 1.0
pontos = 20
aproximacao = regra_do_retangulo(f, a, b, pontos)
print("Aproximação pela Regra do Retângulo = ", round(aproximacao, digits=4), "\n")

# Valor real = 1.71828 (aproximadamente)

Aproximação pela Regra do Retângulo = 1.8116


In [180]:
#Exemplo 3
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (x^3+5x-4) entre os pontos x = 0 e x = 5 ; usando 150 pontos

f(x) = (x^3)+5x-4
a, b = 0.0, 5.0
pontos = 150
aproximacao = regra_do_retangulo(f, a, b, pontos)
print("Aproximação pela Regra do Retângulo = ", round(aproximacao, digits=4), "\n")

# Valor real = 198.75

Aproximação pela Regra do Retângulo = 201.1236


### Função Regra do Trapézio

In [181]:
""" 
Objetivo da função: Encontrar uma aproximação para a integral da função desejada através de 
                    retas ligando os n pontos que temos da função (formando trapézios entre as
                    retas e o eixo x e aproximando a integral final com as áreas desses 
                    trapézios).
Entrada: função, ponto a (ponto inicial -> x0), ponto b (ponto final -> xn), n (número de 
         pontos que usaremos).
Saída: aproximação para o valor da integral de f entre os pontos [a,b].
"""

function regra_do_trapezio(f, a, b, n; T=Float64)
    h = (b - a) / n
    S = zero(T)
    for i = 1:n-1
        x = a + i * h
        S += f(x)
    end
    S = f(a) + 2S + f(b)
    return h * S / 2
end

regra_do_trapezio (generic function with 1 method)

#### Exemplos:

In [182]:
# Exemplo 1
# Imagine que queremos determinar uma aproximação para a área limitada pelo círculo
# (x^2) + (y^2) = 4 no primeiro quadrante, com 10 pontos

f(x) = sqrt(4-x^2)
a, b = 0.0, 2.0
pontos = 10
aproximacao = regra_do_trapezio(f, a, b, pontos)
print("Aproximação pela Regra do Trapézio = ", round(aproximacao, digits=4), "\n")

# Valor real = 3.14159 (aproximadamente)
# Aproximação que tínhamos usando a regra do retângulo com a mesma quantidade de pontos (10) = 3.3045

Aproximação pela Regra do Trapézio = 3.1045


In [183]:
# Exemplo 2
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (e^x) entre os pontos x = 0 e x = 1 ; usando 20 pontos

f(x) = exp(x)
a, b = 0.0, 1.0
pontos = 20
aproximacao = regra_do_trapezio(f, a, b, pontos)
print("Aproximação pela Regra do Trapézio = ", round(aproximacao, digits=4), "\n")

# Valor real = 1.71828 (aproximadamente)
# Aproximação que tínhamos usando a regra do retângulo com a mesma quantidade de pontos (20) = 1.8116

Aproximação pela Regra do Trapézio = 1.7186


In [184]:
#Exemplo 3
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (x^3+5x-4) entre os pontos x = 0 e x = 5 ; usando 150 pontos

f(x) = (x^3)+5x-4
a, b = 0.0, 5.0
pontos = 150
aproximacao = regra_do_trapezio(f, a, b, pontos)
print("Aproximação pela Regra do Trapézio = ", round(aproximacao, digits=4), "\n")

# Valor real = 198.75
# Aproximação que tínhamos usando a regra do retângulo com a mesma quantidade de pontos (150) = 201.1236

Aproximação pela Regra do Trapézio = 198.7569


### Função Regra de Simpson

In [185]:
"""
Objetivo da função: Encontrar uma aproximação para a integral da função desejada através de 
                    polinômios de grau 2 ligando os n pontos que temos da função (aproximando 
                    a integral final com as áreas entre esses polinômios quadráticos e o eixo
                    x).
Entrada: função, ponto a (ponto inicial -> x0), ponto b (ponto final -> xn), n (número de 
         pontos que usaremos).
Saída: aproximação para o valor da integral de f entre os pontos [a,b].
"""

function regra_de_simpson(f, a, b, n; T=Float64)
    if n % 2 != 0
        error("n deve ser par")
    end
    h = (b - a) / n
    S = zero(T)
    for i = 1:2:n-1
        x = a + i * h
        S += 4 * f(x)
    end
    for i = 2:2:n-2
        x = a + i * h
        S += 2* f(x)
    end
    S += f(a) + f(b)
    return S * h / 3
end

regra_de_simpson (generic function with 1 method)

#### Exemplos:

In [186]:
# Exemplo 1
# Imagine que queremos determinar uma aproximação para a área limitada pelo círculo
# (x^2) + (y^2) = 4 no primeiro quadrante, com 10 pontos

f(x) = sqrt(4-x^2)
a, b = 0.0, 2.0
pontos = 10
aproximacao = regra_de_simpson(f, a, b, pontos)
print("Aproximação pela Regra de Simpson = ", round(aproximacao, digits=4), "\n")

# Valor real = 3.14159 (aproximadamente)
# Aproximação que tínhamos usando a regra do retângulo com a mesma quantidade de pontos (10) = 3.3045
# Aproximação que tínhamos usando a regra do trapézio com a mesma quantidade de pontos (10) = 3.1045

Aproximação pela Regra de Simpson = 3.127


In [187]:
# Exemplo 2
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (e^x) entre os pontos x = 0 e x = 1 ; usando 10 pontos

f(x) = exp(x)
a, b = 0.0, 1.0
pontos = 10
aproximacao = regra_de_simpson(f, a, b, pontos)
print("Aproximação pela Regra de Simpson = ", round(aproximacao, digits=4), "\n")

# Valor real = 1.71828 (aproximadamente)
# Aproximação que tínhamos usando a regra do retângulo com o dobro de pontos (20) = 1.8116
# Aproximação que tínhamos usando a regra do trapézio com o dobro de pontos (20) = 1.7186

Aproximação pela Regra de Simpson = 1.7183


In [188]:
# Exemplo 3
# Imagine que queremos determinar uma aproximação para a área limitada pela função
# (x^3+5x-4) entre os pontos x = 0 e x = 5 ; usando 30 pontos

f(x) = (x^3)+5x-4
a, b = 0.0, 5.0
pontos = 30
aproximacao = regra_de_simpson(f, a, b, pontos)
print("Aproximação pela Regra de Simpson = ", round(aproximacao, digits=4), "\n")

# Valor real = 198.75
# Aproximação que tínhamos usando a regra do retângulo com muito mais pontos (150) = 201.1236
# Aproximação que tínhamos usando a regra do trapézio com muito mais pontos (150) = 198.7569

Aproximação pela Regra de Simpson = 198.75


### Função Achar 1 Ponto Crítico de uma Função $F(x)$

In [72]:
"""
Objetivo:
Entrada:
Saída:
"""
function achar_um_ponto_critico_de_uma_função(derivada_funcao, precisao, a, b)
    ponto_critico = bissecao(derivada_funcao(x), precisao, a, b)
    return ponto_critico
end

achar_um_ponto_critico_de_uma_função

#### Exemplos:

In [73]:
# Exemplo 1
# Suponha que queremos saber um ponto crítico da função sen(x) no intervalo [0.1, 5]

f(x) = sin(x)
derivada_f(x) = cos(x)
a = 0.1
b = 5
precisao = 1e-8
achar_um_ponto_critico_de_uma_função(derivada_f, precisao, a, b)

3.141592650860548

In [74]:
# Exemplo 2
# Suponha que queremos saber um ponto crítico da função (x^3)- 4x + 1 no intervalo [-3, 0]

f(x) = (x^3) + 2x
derivada_f(x) = (3x^2 + 2)
a = -2
b = 2
precisao = 1e-16
achar_um_ponto_crítico_de_uma_função(derivada_f, precisao, a, b)

0.0

In [82]:
# Exemplo 3
# Suponha que queremos saber um ponto crítico da função (e^x) - 1 no intervalo [0, 4]

f(x) = (ℯ^x) - 1
derivada_f(x) = (ℯ^x)
a = -4
b = 2
precisao = 1e-16
achar_um_ponto_crítico_de_uma_função(derivada_f, precisao, a, b)

4.440892098500626e-16

### [FALTA] Função Achar 1 Ponto Crítico de uma Função $F(x,y)$

In [86]:
"""
Objetivo:
Entrada:
Saída:
"""
function achar_um_ponto_critico_de_uma_função_de_2_variávies(gradiente_da_funcao,intervalo) #desafio
    ponto_critico = gradiente_descendente
    return ponto_critico
end

achar_um_ponto_critico_de_uma_função_de_2_variávies

#### Exemplos:

In [35]:
# Exemplo 1

In [36]:
# Exemplo 2

In [37]:
# Exemplo 3

### Função Lagrange 2D

In [71]:
"""
RESUMO: Recebe uma série de pontos (x,y,z) e um ponto especifico (xp, yp).
        Descobre uma funcao que interpola os pontos (x,y,z) e descobre a imagem
        de (xp, yp) nessa funcao.
        O grau do polinomio vai ser definido pelo numero de pontos passados.

ENTRADA: Vetor x, Vetor y, Vetor z, Float xp, Float yp

SAIDA: Float zp (imagem de (xp,yp) na funcao que interpola os pontos (x,y,z))
"""
function lagrange_2D(x, y, z, xp, yp)
    # pega o numero de pontos
    n = length(x)
    # resultado da interpolação
    zp = 0
   
    # Implementando lagrange 2D
    for i = 1:n
        p = 1.0
        
        # vetor de valores ja utilizados (para nao repetir no método de Lagrange)
        usados = []
        
        # calcula a parte dos x's
        for j = 1:n
            if x[i] != x[j] && x[j] ∉ usados
                p *= (xp - x[j])/(x[i] - x[j])
            end
            append!(usados, x[j])
        end
        
        # calcula a parte dos y's
        usados = []
        for j = 1:n
            if y[i] != y[j] && y[j] ∉ usados
                p *= (yp - y[j])/(y[i] - y[j])
            end
            append!(usados, y[j])
        end
        zp += p * z[i]
    end
    return zp
end

lagrange_2D

#### Exemplos:

In [72]:
# Exemplo 1
x = [1., 1, 3, 3]
y = [2., 4, 2, 4]
z = [800., 400, 600, 500]
lagrange_2D(x, y, z, 1.0, 2.0)

800.0

In [74]:
# Exemplo 2
n = 5
x = rand(n)
y = rand(n)
z = rand(n)
lagrange_2D(x, y, z, 0.5, 1.51)

-62.624174323626214

In [75]:
# Exemplo 3
x = [1., 18, 37, 93, 112, 193]
y = [1., 25, 56, 170, 250, 401]
z = [8., 17, 50, 121.1, -10, -300]
lagrange_2D(x, y, z, 0.1, 2.4)

7.425253806675609